# Investigar en Wikipedia

In [10]:
queries = ['chocloate'] 

In [11]:
import wikipediaapi
import wikipedia
from fuzzywuzzy import process

In [12]:
def search_wikipedia(query, language='en', top_n=5):
    wiki = wikipediaapi.Wikipedia('MyProjectName (merlin@example.com)', language)
    search_results = wiki.search(query)

    if not search_results:
        return f"No results found for '{query}'."
    
    matches = process.extract(query, search_results, limit=top_n)
    results = []
    for title, score in matches:
        page = wiki.page(title)
        if page.exists():
            results.append({
                'title': title,
                'summary': page.summary[:200] + '...' if len(page.summary) > 200 else  page.summary,
                'score': score
            }) 

    return results

In [13]:
def search_wikipedia(query, top_n=5):
    try:
        search_results = wikipedia.search(query)

        if not search_results:
            return f"No results found for '{query}'."
        

        results = []

        matches = process.extract(query, search_results, limit=top_n)

        for title, score in matches:
            try:
                page = wikipedia.page(title)
                results.append({
                    'title': title,
                    'summary': page.summary[:200] + '...' if len(page.summary) > 200 else page.summary,
                    'score': score 
                })
            except wikipedia.DisambiguationError as e:
                results.append({
                    'title': title,
                    'summary': "Disambiguation page. Options: " + ", ".join(e.options[:5] ) + "...",
                    'score': score
                })
            except wikipedia.PageError:
                results.append({
                    'title': title,
                    'summary': "Page not found.",
                    'score': score
                } )
        return results
    except Exception as e:
        return f"An error occurred: {e}"


In [19]:
query = "chocolate"
results = search_wikipedia(query)
print(results)

[{'title': 'Chocolate', 'summary': 'Chocolate is a food made from roasted and ground cocoa beans that can be a liquid, solid, or paste, either on its own or as a flavoring in other foods. The cacao tree has been used as a source of food...', 'score': 100}, {'title': 'Hot chocolate', 'summary': 'Hot chocolate, also known as hot cocoa or drinking chocolate, is a heated drink consisting of shaved or melted chocolate or cocoa powder, heated milk or water, and usually a sweetener. It is often gar...', 'score': 95}, {'title': 'White chocolate', 'summary': 'White chocolate is a form of chocolate made of cocoa butter, sugar and milk. Unlike milk and dark chocolate, it does not contain cocoa solids, which darken the chocolate. White chocolate has an ivory ...', 'score': 90}, {'title': 'Dubai chocolate', 'summary': "Dubai chocolate is a chocolate bar with a filling made of kadayif (knafeh) and pistachio. It was first created by Fix Dessert Chocolatier in Dubai in 2022, branded as Can't Get Knafe

In [23]:
results = search_wikipedia(query)

# Print the section titles of the first search result
page = wikipedia.page(results[0]['title'])
print(page.sections)

[]


In [4]:
%%python -m spacy download en_core_web_sm


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [7]:


from pydantic import BaseModel
import spacy
import wikipediaapi
import re
from typing import List, Optional
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.chunk import tree2conlltags

# Initialize the NLP tool (spaCy or NLTK for entity extraction)
nlp = spacy.load("en_core_web_sm")  # You can switch to a different model if needed
wiki_wiki = wikipediaapi.Wikipedia('MyProjectName', 'en')

# Step 1: Define the Pydantic data models

class QueryRequest(BaseModel):
    query: str

class QueryResponse(BaseModel):
    titles: List[str]

# Step 2: Function to extract entities from the query using NLP
def extract_entities(query: str) -> List[str]:
    doc = nlp(query)
    entities = [ent.text for ent in doc.ents]
    return entities

# Step 3: Function to search Wikipedia for entities and return relevant titles
def search_wikipedia_for_entities(entities: List[str]) -> List[str]:
    titles = []
    
    for entity in entities:
        search_results = wiki_wiki.search(entity, results=3)  # Limiting to top 3 results
        titles.extend(search_results)  # Add titles to the list
    
    return titles

# Step 4: Chain function
def process_query(query: str) -> QueryResponse:
    # Extract entities
    entities = extract_entities(query)
    
    # If no entities found, return an empty response
    if not entities:
        return QueryResponse(titles=[])
    
    # Search Wikipedia for entities
    titles = search_wikipedia_for_entities(entities)
    
    # Return the response with titles
    return QueryResponse(titles=titles)

In [10]:
from pydantic import BaseModel
import spacy
import wikipedia
from typing import List, Optional

# Initialize the NLP tool (spaCy)
nlp = spacy.load("en_core_web_sm")  # You can switch to a different model if needed

# Step 1: Define the Pydantic data models

class QueryRequest(BaseModel):
    query: str

class QueryResponse(BaseModel):
    titles: List[str]

# Step 2: Function to extract entities from the query using NLP
def extract_entities(query: str) -> List[str]:
    doc = nlp(query)
    entities = [ent.text for ent in doc.ents]
    return entities

# Step 3: Function to search Wikipedia for entities and return relevant titles
def search_wikipedia_for_entities(entities: List[str]) -> List[str]:
    titles = []
    
    for entity in entities:
        search_results = wikipedia.search(entity, results=3)  # Limiting to top 3 results
        titles.extend(search_results)  # Add titles to the list
    
    return titles

# Step 4: Chain function
def process_query(query: str) -> QueryResponse:
    # Extract entities
    entities = extract_entities(query)
    
    # If no entities found, return an empty response
    if not entities:
        return QueryResponse(titles=[])
    
    # Search Wikipedia for entities
    titles = search_wikipedia_for_entities(entities)
    
    # Return the response with titles
    return QueryResponse(titles=titles)

# Example usage
query = "Mercury is the smallest planet in the Solar System."
response = process_query(query)

# Displaying the result
print(response)


titles=['Mercury', 'Mercury (planet)', 'Freddie Mercury', 'Solar System', 'Formation and evolution of the Solar System', 'Fictional planets of the Solar System']
